In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import numpy as np
import nltk
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark.sql.functions as func
from pyspark.sql.types import BooleanType,IntegerType

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1558416731067_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.default.parallelism","20") \
    .config("spark.executor.memory","16g") \
    .config("spark.driver.memory","16g") \
    .appName("Stage Two") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

VBox()

In [3]:
# Load the orginal dataset
rawdata = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
raw = spark.read.csv(rawdata,header=True,sep='\t')

VBox()

In [4]:
# Print dataset columns schema
raw.printSchema()

VBox()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

In [5]:
# Filter out the reviews less than two sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
valid_sentences = lambda paragraph: (paragraph != None) and (len(tokenizer.tokenize(paragraph)) >= 2)
udf_sentences = udf(valid_sentences,BooleanType())
filtersentences = raw.filter(udf_sentences(raw.review_body))

VBox()

In [7]:
# Filter out the reviews published by users with less than median number of reviews published
median_revs_of_user = raw.groupby('customer_id').count().approxQuantile('count',[0.5],0.05)
median_customer = median_revs_of_user[0]
ReviewByUser = raw.groupby('customer_id').count().cache()
CustomerID = ReviewByUser.filter(ReviewByUser['count'] <= median_customer)

VBox()

In [9]:
# Filter out the reviews from products with less than median number of reviews received
median_revs_of_product = raw.groupby('product_id').count().approxQuantile('count',[0.5],0.05)
median_product = median_revs_of_product[0]
ReviewByProduct = raw.groupby('product_id').count().cache()
ProductID = ReviewByProduct.filter(ReviewByProduct['count'] <= median_product)

VBox()

In [21]:
# Converting to rdd to filter out the unwanted data and save the result as dataframe
customerid = CustomerID.rdd
productid = ProductID.rdd
filterrdd = filtersentences.rdd.map(lambda x: (x.customer_id, (x.product_id, x.review_id, x.star_rating, x.review_body))) \
            .subtractByKey(customerid).map(lambda x: (x[1][0],(x[0],x[1][1],x[1][2],x[1][3]))) \
            .subtractByKey(productid).map(lambda x: (x[1][0],x[0],x[1][1],x[1][2],x[1][3])) 
filteredata = spark.createDataFrame(filterrdd, ['customer_id','product_id','review_id','star_rating','review_body'])

VBox()

In [22]:
# Define the udf to find the number of the sentences published for each review
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
num_sentences = lambda paragraph: len(tokenizer.tokenize(paragraph)) if (paragraph != None) else 0
udf_sentences_num = udf(num_sentences,IntegerType())

VBox()

In [23]:
# Define udf for finding the median for the given list
median_sentences = lambda val_list : int(np.median(val_list))
udf_median_sentences = udf(median_sentences,IntegerType())

VBox()

In [24]:
# Top 10 users ranked by median number of sentences in the reviews they have published
filteredata.groupBy('customer_id') \
    .agg(func.collect_list(udf_sentences_num(filteredata.review_body)).alias('val_list')) \
    .withColumn('median',udf_median_sentences('val_list')) \
    .sort('median', ascending=False) \
    .limit(10) \
    .show()

VBox()

+-----------+--------------------+------+
|customer_id|            val_list|median|
+-----------+--------------------+------+
|   25628286|          [251, 218]|   234|
|   37118941|               [227]|   227|
|   51865782|           [12, 440]|   226|
|   29580246|               [201]|   201|
|   50595705|[82, 290, 196, 19...|   191|
|   17821650|      [183, 68, 228]|   183|
|   43879820|          [182, 179]|   180|
|   15585529|               [177]|   177|
|   23717536|               [157]|   157|
|   46097534|           [228, 81]|   154|
+-----------+--------------------+------+

In [25]:
# Top 10 products ranked by median number of sentences in the reviews they have received
filteredata.groupBy('product_id') \
    .agg(func.collect_list(udf_sentences_num(filteredata.review_body)).alias('val_list')) \
    .withColumn('median',udf_median_sentences('val_list')) \
    .sort('median', ascending=False) \
    .limit(10) \
    .show()

VBox()

+----------+--------+------+
|product_id|val_list|median|
+----------+--------+------+
|B00LTQ5EVY|   [984]|   984|
|B009SF2GZU|   [321]|   321|
|B000003G29|   [267]|   267|
|B00T7TYTCK|[2, 503]|   252|
|B000BCH5PK|   [209]|   209|
|B0000C0FEW|   [200]|   200|
|B00AP5M4WM|   [160]|   160|
|B009SF2IRG|[308, 7]|   157|
|B008LA8E9K|   [149]|   149|
|B005ZHBBU6|[2, 293]|   147|
+----------+--------+------+